In [1]:
import json
import os, shutil, pandas as pd

#### Filtering Clinical Trails study

In [2]:
parent_dir = os.path.dirname(os.getcwd())
dataset_path_pmc = os.path.join(parent_dir,'dataset','cord-19_2020-07-09','2020-07-09','document_parses','pmc_json')
clinical_path_pmc = os.path.join(parent_dir,'dataset','clinical_trials','pmc')

In [7]:
parent_dir = os.path.dirname(os.getcwd())
dataset_path_pmc = os.path.join(parent_dir,'dataset','cord-19_2020-07-16','2020-07-16','document_parses','pmc_json')
clinical_path_pmc = os.path.join(parent_dir,'dataset','clinical_trials','round_5' ,'pmc')

In [3]:
for pmc in os.listdir(dataset_path_pmc):
    print(pmc)
    data = json.load(open(os.path.join(dataset_path_pmc,pmc)))
    for i in range(len(data['body_text'])):
        if "clinical trial" in data['body_text'][i]['text'].lower() or "randomized controlled trial" in data['body_text'][i]['text'].lower() \
            or 'interventional study' in data['body_text'][i]['text'].lower():
            shutil.copy(src=os.path.join(dataset_path_pmc,pmc),dst=os.path.join(clinical_path_pmc,pmc))
            print('{} copied sucessfully'.format(pmc))
    

PMC7240309.xml.json
PMC7229919.xml.json
PMC7223453.xml.json
PMC7277829.xml.json
PMC7104966.xml.json
PMC7227176.xml.json
PMC3339311.xml.json
PMC7299853.xml.json
PMC6376732.xml.json
PMC6376732.xml.json copied sucessfully
PMC6376732.xml.json copied sucessfully
PMC2744231.xml.json
PMC7211994.xml.json
PMC7129573.xml.json
PMC7361941.xml.json
PMC2923116.xml.json
PMC2923116.xml.json copied sucessfully
PMC2923116.xml.json copied sucessfully
PMC2923116.xml.json copied sucessfully
PMC2923116.xml.json copied sucessfully
PMC2923116.xml.json copied sucessfully
PMC7282020.xml.json
PMC7282020.xml.json copied sucessfully
PMC7282020.xml.json copied sucessfully
PMC7227701.xml.json
PMC7227701.xml.json copied sucessfully
PMC7227701.xml.json copied sucessfully
PMC7227701.xml.json copied sucessfully
PMC7256552.xml.json
PMC7206038.xml.json
PMC6942430.xml.json
PMC7188328.xml.json
PMC7050115.xml.json
PMC7099678.xml.json
PMC7163426.xml.json
PMC7163426.xml.json copied sucessfully
PMC7163426.xml.json copied sucess

KeyboardInterrupt: 

#### Filter metadata file of clinical trials

In [4]:
CT_pmc_id = [file[:-9] for file in os.listdir(clinical_path_pmc)]

In [6]:
len(CT_pmc_id)

9119

In [67]:
df = pd.read_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/cord-19_2020-07-16/2020-07-16/metadata.csv')

/home/maaz-lfd/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3442: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:
df = df.dropna(subset=['pmcid'])

In [10]:
CT_metdata = df[df['pmcid'].isin(CT_pmc_id)]

In [308]:
CT_metdata = CT_metdata.dropna(subset=['doi'])

In [330]:
CT_metdata.to_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/round_5_CT_metdata.csv', index=False)

In [9]:
CT_metdata = pd.read_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/round_5_CT_metdata.csv')

In [10]:
CT_metdata.isna().sum()

Unnamed: 0             0
cord_uid               0
sha                  560
source_x               0
title                  0
doi                    0
pmcid                  0
pubmed_id           1552
license                0
abstract            1282
publish_time           0
authors               61
journal               22
mag_id              9100
who_covidence_id    8547
arxiv_id            9098
pdf_json_files       560
pmc_json_files         0
url                    0
s2_id               2387
dtype: int64

In [337]:
CT_metdata['abstract'].isin(['NA']).sum()

1282

In [329]:
CT_metdata['abstract'] = CT_metdata['abstract'].fillna('NA')

In [316]:
sorted_df = CT_metdata['abstract'].sort_values(key=lambda x: x.str.len(), ascending=True)

#### Adding abstract to pmc json

In [2]:
CT_dt = pd.read_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/round_5_CT_metdata.csv')

In [5]:
CT_dt.isna().sum()

Unnamed: 0             0
cord_uid               0
sha                  560
source_x               0
title                  0
doi                    0
pmcid                  0
pubmed_id           1552
license                0
abstract               0
publish_time           0
authors               61
journal               22
mag_id              9100
who_covidence_id    8547
arxiv_id            9098
pdf_json_files       560
pmc_json_files         0
url                    0
s2_id               2387
dtype: int64

In [4]:
CT_dt['abstract'] = CT_dt['abstract'].fillna("NA")

In [8]:
for index, rows in CT_dt.iterrows():
    data = json.load(open(os.path.join(clinical_path_pmc,rows['pmcid']+'.xml.json')))
    if rows['abstract'] != "NA":
        data['metadata'].update({ 'abstract': rows['abstract']})
        with open(os.path.join(clinical_path_pmc,rows['pmcid']+'.xml.json'),'w') as f:
            json.dump(data, f, indent=4)

In [98]:
for file in remaining:
    data = json.load(open(os.path.join(clinical_path_pmc,file+'.xml.json')))
    abstract = df[df['pmcid']==file]['abstract'].values[0]
    data['metadata'].update({ 'abstract': abstract})
    with open(os.path.join(clinical_path_pmc,file+'.xml.json'),'w') as f:
        json.dump(data, f, indent=4)

In [99]:
without_abstract = []
for filename in os.listdir(clinical_path_pmc):
    if 'abstract' not in json.load(open(os.path.join(clinical_path_pmc,filename)))['metadata']:
        without_abstract.append(filename[:-9])

In [100]:
len(without_abstract)

1282

In [31]:
without_abstract[-2]

'PMC7184506'

In [89]:
CT_dt.loc[181773]

IndexError: single positional indexer is out-of-bounds

In [88]:
CT_dt[CT_dt['pmcid']== 'PMC4631129']

,Unnamed: 0,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id


In [91]:
"PMC4631129" in CT_dt['pmcid'].to_list()

False

In [97]:
pmc = 'PMC1435713'
df[df['pmcid']==pmc]['abstract'].values[0]

'This article presents a systematic framework of 12 essentials, or basic elements, of science-based policy. The 12 essentials are grouped into three categories, or areas, as follows: 1) knowledge generation, which includes credible design, accurate data, sound analysis, and comprehensive synthesis; 2) knowledge exchange, which includes relevant content, appropriate translation, timely dissemination, and modulated release; and 3) knowledge uptake, which includes accessible information, readable message, motivated user, and rewarding outcome.'

In [75]:
df[df['pmcid']== 'PMC1435713']

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
181773,75ytw7ez,0a6e9aa35d1320355bf071879e98aabcacfe2b85,Medline; PMC,Twelve Essentials of Science-based Policy,NaN,PMC1435713,16164820,no-cc,This article presents a systematic framework o...,2005-09-15,"Choi, Bernard C.K",Prev Chronic Dis,NaN,NaN,NaN,document_parses/pdf_json/0a6e9aa35d1320355bf07...,document_parses/pmc_json/PMC1435713.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/16164820/,9896150.0


In [90]:
df.loc[181773]

cord_uid                                                     75ytw7ez
sha                          0a6e9aa35d1320355bf071879e98aabcacfe2b85
source_x                                                 Medline; PMC
title                       Twelve Essentials of Science-based Policy
doi                                                               NaN
pmcid                                                      PMC1435713
pubmed_id                                                    16164820
license                                                         no-cc
abstract            This article presents a systematic framework o...
publish_time                                               2005-09-15
authors                                             Choi, Bernard C.K
journal                                              Prev Chronic Dis
mag_id                                                            NaN
who_covidence_id                                                  NaN
arxiv_id            

In [ ]:
df[df['pmcid']== 'PMC1435713']['abstract'].values

In [101]:
abs_na = CT_dt[CT_dt['abstract']== 'NA']

In [102]:
abs_na_pmc = abs_na['pmcid'].to_list()

In [103]:
set(without_abstract).difference(set(abs_na_pmc))

set()

In [92]:
remaining = set(without_abstract).difference(set(abs_na_pmc))

In [93]:
remaining

{'PMC1435713',
 'PMC2699953',
 'PMC2721300',
 'PMC3347541',
 'PMC3734712',
 'PMC3928693',
 'PMC4631129',
 'PMC4718316',
 'PMC5662269',
 'PMC6500415',
 'PMC7075271',
 'PMC7134243',
 'PMC7141793',
 'PMC7169474',
 'PMC7213070',
 'PMC7213073',
 'PMC7213074',
 'PMC7331973',
 'PMC7348435'}

In [104]:
remaining = ['PMC1435713',
 'PMC2699953',
 'PMC2721300',
 'PMC3347541',
 'PMC3734712',
 'PMC3928693',
 'PMC4631129',
 'PMC4718316',
 'PMC5662269',
 'PMC6500415',
 'PMC7075271',
 'PMC7134243',
 'PMC7141793',
 'PMC7169474',
 'PMC7213070',
 'PMC7213073',
 'PMC7213074',
 'PMC7331973',
 'PMC7348435']

In [105]:
new_df = df[df['pmcid'].isin(remaining)]

In [106]:
new_df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
77938,ra756rn1,NaN,PMC,Pre-clinical toxicity & immunobiological evalu...,NaN,PMC3734712,23852288.0,cc-by-nc-sa,BACKGROUND & OBJECTIVES: Pre-clinical toxicolo...,2013-06-16,"Kumar, B. Dinesh; Kumar, P. Uday; Krishna, T. ...",Indian J Med Res,NaN,NaN,NaN,NaN,document_parses/pmc_json/PMC3734712.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
78041,koaaxltk,3afd5d184312c4501ab9038756e7297e3e87b1d5,PMC,COVID-19 and Ophthalmology: A New Chapter in a...,NaN,PMC7134243,NaN,cc-by,NaN,2020-03-04,"Reviglio, Victor Eduardo; Osaba, Matias; Revig...",Med Hypothesis Discov Innov Ophthalmol,NaN,NaN,NaN,document_parses/pdf_json/3afd5d184312c4501ab90...,document_parses/pmc_json/PMC7134243.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
78042,2qqkoeo9,a4a07b11d77ec96452acb8806b274d7654a72ed4,PMC,COVID-19: Preliminary Clinical Guidelines for ...,NaN,PMC7141793,NaN,cc-by,The zoonotic Severe Acute Respiratory Syndrome...,2020-04-06,"Gharebaghi, Reza; Desuatels, Jordan; Moshirfar...",Med Hypothesis Discov Innov Ophthalmol,NaN,NaN,NaN,document_parses/pdf_json/a4a07b11d77ec96452acb...,document_parses/pmc_json/PMC7141793.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
78050,qz8pffei,27521ddbfb535760a7e4d77ac0cd6a0f3ebfb36a,PMC,COVID-19 in Singapore and Malaysia: Rising to ...,NaN,PMC7169474,NaN,cc-by,With the increasing number of COVID-19 cases a...,2020-04-07,"Tay, K; Kamarul, T; Lok, WY; Mansor, M; Li, X;...",Malays Orthop J,NaN,NaN,NaN,document_parses/pdf_json/27521ddbfb535760a7e4d...,document_parses/pmc_json/PMC7169474.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
78052,qvcbfnbf,15af325f092c29298a36e80431a7837ea6961439,PMC,Review of safety and minimum pricing of nitazo...,NaN,PMC7213070,NaN,cc-by-nc,BACKGROUND: Many treatments are being assessed...,2020-04-30,"Pepperrell, Toby; Pilkington, Victoria; Owen, ...",NaN,NaN,NaN,NaN,document_parses/pdf_json/15af325f092c29298a36e...,document_parses/pmc_json/PMC7213070.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
78054,wcdktaed,7ca9d3d5d23c8912a57db0e41f0a6f23a05b3408,PMC,A review of the safety of favipiravir – a pote...,NaN,PMC7213073,NaN,cc-by-nc,BACKGROUND: Repurposing broad-spectrum antivir...,2020-04-30,"Pilkington, Victoria; Pepperrell, Toby; Hill, ...",NaN,NaN,NaN,NaN,document_parses/pdf_json/7ca9d3d5d23c8912a57db...,document_parses/pmc_json/PMC7213073.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
78055,3rblzyry,9c6c4df1d158316804f7e4fedc269307af8e4f33,PMC,Minimum costs to manufacture new treatments fo...,NaN,PMC7213074,NaN,cc-by-nc,INTRODUCTION: ‘Repurposing’ existing drugs to ...,2020-04-30,"Hill, Andrew; Wang, Junzheng; Levi, Jacob; Hea...",NaN,NaN,NaN,NaN,document_parses/pdf_json/9c6c4df1d158316804f7e...,document_parses/pmc_json/PMC7213074.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
78073,w9ta5gs1,06012e37c48504c6782326602f07954918d74f79,PMC,Whole Genome Analysis and Targeted Drug Discov...,NaN,PMC7331973,NaN,cc-by,A novel coronavirus designated as SARS-CoV-2 i...,2020-03-30,"Manikyam, Hemanth Kumar; Joshi, Sunil K",J Pharm Drug Res,NaN,NaN,NaN,document_parses/pdf_json/06012e37c48504c678232...,document_parses/pmc_json/PMC7331973.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
78074,nb2sxfax,6536d3bd360cc4115bc93d1ead1eeec262026acc,PMC,Traditional Healing Practices Involving Psycho...,NaN,PMC7348435,NaN,cc-by-nc,NaN,2020-06-16,"Bouso, José Carlos; Sánchez-Avilés, Constanza",Health Hum Rights,NaN,NaN,NaN,document_parses/pdf_json/6536d3bd360cc4115bc93...,document_parses/pmc_json/PMC7348435.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
145397,rjgpc41r,7e4f3fdf70d647d0530a94f12f90f0927fbc1ff5,Medline; PMC,Biological targets for isatin and its analogue...,NaN,PMC2721300,19707325,no-cc,Isatin 

In [107]:
updated = pd.concat([CT_dt, new_df])

In [109]:
updated.to_csv('/home/maaz-lfd/Maaz/Thesis/Thesis/dataset/clinical_trials/round_5/round_5_CT_metdata.csv')